In [30]:
from dash import Dash, html, dcc, Input, Output  # pip install dash
import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np

import altair as alt                        # pip install altair

import matplotlib.pyplot as plt             # pip install matplotlib
import mpld3                                # pip install mpld3

from bokeh.plotting import figure           # pip install Bokeh
from bokeh.resources import CDN
from bokeh.embed import file_html
from bokeh.models import ColumnDataSource

# bring data into app
casetas = pd.read_excel('Casetas TDR.xlsx')
tlvia = pd.read_excel('Compilado televía 2.xlsx')
# print(cars.columns)

tlcount = tlvia[['TAG', 'Importe']].groupby('TAG').count()
cascount = casetas[['TAG', 'Costo Caseta']].groupby('TAG').count()
tlsum = tlvia[['TAG', 'Importe']].groupby('TAG').sum()
cassum = casetas[['TAG', 'Costo Caseta']].groupby('TAG').sum()
diffcount =  pd.DataFrame(tlcount['Importe'] - cascount['Costo Caseta'])
diffsum = pd.DataFrame(tlsum['Importe'] - cassum['Costo Caseta'])
diffsum = diffsum[diffsum != 0]
diffcount = diffcount[diffcount != 0]
diff = pd.merge(diffcount, diffsum, left_index=True, right_index=True)
diff.rename(columns = {'0_x':'Dif. Casetas pagadas', '0_y':'Dif. Total pagado'}, inplace = True)
tlvia.set_index('TAG', inplace=True)
diff2 = pd.merge(diff, tlvia, left_index=True, right_index=True)
diff2 = diff2[['#Unidad', 'Dif. Casetas pagadas', 'Dif. Total pagado']].dropna()
diff2.drop_duplicates(inplace=True)
diff2['#Unidad'] = diff2['#Unidad'].astype(str)
diff2.set_index('#Unidad', inplace=True)

In [43]:
# Define color palette
colors = {
    'color1': '#023059',
    'color2': '#023859',
    'color3': '#023859',
    'color4': '#F2AE30',
    'color5': '#0D0D0D',
    'color6': '#F2F2F2'}

# Define tu paleta de colores
colors1 = ['#D1EDFF','#96CFF2', '#D9BA71', '#F2AE30', '#5095BF', '#046EB0', '#1F628C', '#035487',
          '#593F02', '#593502', '#233059', '#021C59', '#012136']

In [32]:
import plotly.express as px

# Set up Dash app
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Funciones y cosas
ddown = dcc.Dropdown(id='mydropdown',
        value=['Dif. Total pagado'],
        options=[{'label': i, 'value': i} for i in ['Dif. Casetas pagadas', 'Dif. Total pagado']])

graph0 = dcc.Graph(id="Barras")  

# Set up the page layout
app.layout = html.Div([ddown, graph0])

@app.callback(
    Output('Barras', 'figure'),
    Input('mydropdown', 'value'))

def bar_chart(spending):
    fig = px.bar(
        diff2,
        x=diff2.index,
        y=spending,
        barmode="group",
        title="Diferencia en casetas por unidad",
    )
    return fig

#if __name__ == '__main__':
#    app.run_server(debug=False, port='8004')


In [44]:
# Configurar la aplicación Dash
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Crear el gráfico de pastel con colores personalizados
graph1 = dcc.Graph(figure=px.pie(diff2, 
             values='Dif. Total pagado',
             hole=.45,
             names=diff2.index, 
             title='Distribucion de diferencias en total pagado en Televia vs contabilidad - Enero',
             color_discrete_sequence=colors1))


# Configurar el layout de la página
app.layout = html.Div([graph1]) 

#if __name__ == '__main__':
#    app.run_server(debug=True, port='8004')

In [45]:
# Datos diarios de pagos de casetas por unidad

tlvia1 = tlvia

# Primero simplifico el formato de fecha para que exista una version sin la hora especifica

tlvia1['Fecha'] = pd.to_datetime(tlvia1["Fecha"].dt.strftime('%Y-%m-%d'))

# Procedo a agrupar por unidad y fecha, para hacer una suma de pagos de casetas por dia

tlvia1[['#Unidad', 'Fecha', 'Importe']].groupby(['Fecha', '#Unidad']).sum()

Importe
Fecha      #Unidad         
2024-01-01 1715         250
           1721        1038
           1729         500
           1738        1245
           1746         660
...                     ...
2024-01-31 1816        1245
           1817        1641
           1819        1641
           1824         396
           1826        1245

[629 rows x 1 columns]

In [50]:
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

ddown1 = dcc.Dropdown(
        id='unidad-dropdown',
        options=[{'label': i, 'value': i} for i in tlvia1['#Unidad'].unique()],
        value=tlvia1['#Unidad'].unique()[0])

graph2 = dcc.Graph(id='line-chart')

app.layout = html.Div([ddown1, graph2])

@app.callback(
    Output('line-chart', 'figure'),
    [Input('unidad-dropdown', 'value')]
)
def update_line_chart(unidad):
    df_filtered = tlvia1[tlvia1['#Unidad'] == unidad]
    fig = px.bar(df_filtered, x='Fecha', y='Importe', title='Pagos de casetas por día')
    fig.update_traces(marker_color=colors1[3])  # type: ignore # set the color of the bars
    return fig

#if __name__ == '__main__':
#    app.run_server(debug=True, port='8004')

In [52]:
from dash import dash_table
from dash.dash_table.Format import Group

# Tabla de importes duplicados en Televía

tlvia = pd.read_excel('Compilado televía 2.xlsx')
tlvia['duplicado'] = tlvia.duplicated(subset=['#Unidad', 'Fecha'], keep=False).astype(int)
tdup = tlvia[['Fecha', '#Unidad', 'Entrada', 'Salida', 'Importe']][tlvia['duplicado'] == 1]

app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

ddown2 = dcc.Dropdown(
        id='unidad-dropdown',
        options=[{'label': i, 'value': i} for i in tdup['#Unidad'].unique()],
        value=tdup['#Unidad'].unique()[0])

table = dash_table.DataTable(
        id='table',
        columns=[{"name": i, "id": i} for i in tdup.columns],
        data=tdup.to_dict('records'),
        style_data={'backgroundColor': 'rgb(248, 248, 248)',
                    'color':'black',
                    'fontSize': '12px',  # change the font size to 20px
                    'textAlign': 'left'
                    },
        style_cell={'backgroundColor': 'rgba(2,48,89,255)',
                    'color':'white',
                    'fontSize': '20px',  # change the font size to 20px
                    'textAlign': 'center',
    })

app.layout = html.Div([ddown1, table])

@app.callback(
    Output('table', 'data'),
    Input('unidad-dropdown', 'value'))
def update_table(unidad):
    filtered_df = tdup[tdup['#Unidad'] == unidad]
    return filtered_df.to_dict('records')

#if __name__ == '__main__':
#    app.run_server(debug=True, port='8004')


## Compilado no. 1

In [53]:
#Generamos variables para las tajetas de televia y casetas pagadas  (TDR)

#Contavilizamos el numero de filas del dataset tlvia
caspag = tlvia.Entrada.count()

#Sumamos el importe de las casetas pagadas (TDR)
casimpTDR = casetas['Costo Caseta'].sum()

#Sumamos el importe de las casetas pagadas (Televia)
casimpTV = tlvia['Importe'].sum()


In [56]:
import base64

# Set up Dash app
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

test_png = 'logotdr.png'
test_base64 = base64.b64encode(open(test_png, 'rb').read()).decode('ascii')


tarjetas = dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H4(" # Casetas Pagadas (Televia)", className="card-title"),
                    html.H2(f"{caspag}",
                            className="card-text"),
                ])
            ])
        ]),

        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H4("Importe Caseta Pagadas (TDR)", className="card-title"),
                    html.H2(f"${casimpTDR:,.2f}",
                            className="card-text"),
                ])
            ])
        ]),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H4("Importe Caseta Pagadas (Televia)", className="card-title"),
                    html.H2(f"${casimpTV:,.2f}",
                            className="card-text"),
                ])
            ])
        ])
    ])

# Set up the page layout
app.layout = html.Div([
    html.Div(
        style={
            'backgroundColor': colors['color1'],
            'padding': '5px',
            'textAlign': 'center'},
        children=[html.Img(src='data:image/png;base64,{}'.format(test_base64),
                           style={'height':'105px',
                                  'width':'256px'})
        ]
    ),
 
    dcc.Tabs(
        id='tabs',
        className= 'dbc',
        value='tab-1',
        style={'backgroundColor': colors['color6']},
        children=[
            dcc.Tab(
                label = 'Overview',
                value = 'tab-1',
                children = [
                    html.Br(),
                    
                    dbc.Row([
                        tarjetas, 
                    ]),
                    html.Br(),
                    dbc.Row([
                        dbc.Col([
                            dbc.Card([
                                ddown, 
                                graph0,
                            ])
                        ]),
                        html.Br(),
                        dbc.Col([
                            dbc.Card([ 
                                graph1
                            ])
                        ])
                    ])
                ]
            ),
            dcc.Tab(
                label = 'Pagos por unidad',
                value = 'tab-2',
                children = [
                    html.Br(),
                    dbc.Row([
                        dbc.Col([
                            html.H4("Selecciona la unidad: ", className="text-end"), 


                        ]), 
                        dbc.Col([
                                ddown2,
                        ])
                    ]),
                    html.Br(),
                    dbc.Row([
                        dbc.Card([
                            dbc.Col([
                                dbc.Card([

                                    graph2
                                ])
                            ])
                        ])
                    ]),
                    html.Br(),
                    html.Br(),
                    dbc.Row([
                        dbc.Card([
                        dbc.Col([
                                html.H4("Listado de pagos duplicados"),  
                                html.Br(),
                                table
                            ])
                        ])
                    ])
                ]
            ),
        ]
    )
])



#Definimos la funcion para el grafico de barras
@app.callback(
    Output('Barras', 'figure'),
    Input('mydropdown', 'value'))

def bar_chart(spending):
    fig = px.bar(
        diff2,
        x=diff2.index,
        y=spending,
        barmode="group",
        title="Diferencia en casetas por unidad",
    )
    fig.update_traces(marker_color=colors1[6])  # set the color of the bars
    return fig

@app.callback(
    [Output('line-chart', 'figure'),
    Output('table', 'data')],
    [Input('unidad-dropdown', 'value')])

def content(unidad):
    df_filtered = tlvia1[tlvia1['#Unidad'] == unidad]
    fig = px.bar(df_filtered, x='Fecha', y='Importe', title='Pagos de casetas por día')
    fig.update_traces(marker_color=colors1[6])  # set the color of the bars

    # Update table
    filtered_df = tdup[tdup['#Unidad'] == unidad]
    table_data = filtered_df.to_dict('records')

    return fig, table_data


if __name__ == '__main__':
    app.run_server(debug=False, port='8005')

In [39]:
# Guardar en html

#!jupyter nbconvert --to html Reto_Dashboard.ipynb